<a href="https://colab.research.google.com/github/zyzsdsg/Machine-Learning/blob/main/BasicRNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [19]:
import pandas as pd
import numpy as np
import torch
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import torch.optim as optim

In [39]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [3]:
vix_data = pd.read_csv("/content/drive/MyDrive/VIX/VIX_History.csv")
vix_data["DATE"] = pd.to_datetime(vix_data["DATE"])
vix_data = vix_data.sort_values('DATE')
close_prices = vix_data['CLOSE'].values
vix_data.head()

,DATE,OPEN,HIGH,LOW,CLOSE
0,1990-01-02,17.24,17.24,17.24,17.24
1,1990-01-03,18.19,18.19,18.19,18.19
2,1990-01-04,19.22,19.22,19.22,19.22
3,1990-01-05,20.11,20.11,20.11,20.11
4,1990-01-08,20.26,20.26,20.26,20.26


In [28]:
def create_dataset(data, window_size=10):
    X, y = [], []
    for i in range(len(data) - window_size - 1):
        x_seq = data[i:i+window_size]
        next_day = data[i + window_size]
        today = data[i + window_size - 1]
        label = 1 if next_day > today else 0
        X.append(x_seq)
        y.append(label)
    return np.array(X), np.array(y)

X, y = create_dataset(close_prices, window_size=10)
print(X)
print(y)

[[17.24 18.19 19.22 ... 20.05 24.64 26.34]
 [18.19 19.22 20.11 ... 24.64 26.34 24.18]
 [19.22 20.11 20.26 ... 26.34 24.18 24.16]
 ...
 [14.77 15.37 15.35 ... 19.43 19.1  21.13]
 [15.37 15.35 15.27 ... 19.1  21.13 19.63]
 [15.35 15.27 15.66 ... 21.13 19.63 22.78]]
[0 0 1 ... 0 1 1]


In [29]:
X = np.expand_dims(X, axis=2)  # [N, 10, 1]
X_tensor = torch.tensor(X, dtype=torch.float32)
y_tensor = torch.tensor(y, dtype=torch.float32).unsqueeze(1)

In [30]:
X_train, X_test, y_train, y_test = train_test_split(X_tensor, y_tensor, test_size=0.2, random_state=42)


In [31]:
import torch.nn as nn
class VIXRNNPredictor(nn.Module):
    def __init__(self, input_size=1, hidden_size=64, num_layers=5):
        super(VIXRNNPredictor, self).__init__()
        self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        out, _ = self.rnn(x)
        out = out[:, -1, :]
        out = self.fc(out)
        return self.sigmoid(out)

model = VIXRNNPredictor()


In [40]:
model = VIXRNNPredictor().to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)
batch_size = 10
epochs = 10


In [41]:


for epoch in range(epochs):
    permutation = torch.randperm(X_train.size(0))
    total_loss = 0
    for i in range(0, X_train.size(0), batch_size):
        indices = permutation[i:i+batch_size]
        batch_x, batch_y = X_train[indices], y_train[indices]
        batch_x = batch_x.to(device)
        batch_y = batch_y.to(device)

        optimizer.zero_grad()
        outputs = model(batch_x)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss: {total_loss:.4f}")


Epoch 1, Loss: 184.8608
Epoch 2, Loss: 184.7624
Epoch 3, Loss: 188.2006
Epoch 4, Loss: 184.7391
Epoch 5, Loss: 184.8180
Epoch 6, Loss: 183.8467
Epoch 7, Loss: 184.4850
Epoch 8, Loss: 186.6578
Epoch 9, Loss: 185.0532
Epoch 10, Loss: 184.7908


In [42]:
with torch.no_grad():
    print("Day | Predicted | Actual | Probability")
    print("-----------------------------------------")

    correct = 0  # Counter for correct predictions
    total = 100  # Number of days to evaluate

    for i in range(-110, -10):  # Loop over the last 100 predictions
        last_10 = close_prices[i:i+10]  # Past 10 days
        true_today = close_prices[i+9]
        true_tomorrow = close_prices[i+10]

        label = 1 if true_tomorrow > true_today else 0  # Ground truth

        scaled = scaler.transform([last_10])           # shape: [1, 10]
        input_tensor = torch.tensor(scaled, dtype=torch.float32)
        input_tensor = input_tensor.view(1, 10, 1)      # shape: [1, 10, 1]
        input_tensor = input_tensor.to(device)
        prob = model(input_tensor).item()              # get predicted probability
        pred = 1 if prob > 0.5 else 0

        pred_text = "Up" if pred == 1 else "Down"
        label_text = "Up" if label == 1 else "Down"

        if pred == label:
            correct += 1

        print(f"Day {i+111:>3} | Predicted: {pred_text:<4} | Actual: {label_text:<4} | Prob: {prob:.4f}")

    accuracy = correct / total
    print("\nOverall Accuracy: {:.2f}%".format(accuracy * 100))


Day | Predicted | Actual | Probability
-----------------------------------------
Day   1 | Predicted: Down | Actual: Up   | Prob: 0.4887
Day   2 | Predicted: Down | Actual: Down | Prob: 0.4887
Day   3 | Predicted: Down | Actual: Down | Prob: 0.4887
Day   4 | Predicted: Down | Actual: Down | Prob: 0.4887
Day   5 | Predicted: Down | Actual: Up   | Prob: 0.4887
Day   6 | Predicted: Down | Actual: Down | Prob: 0.4887
Day   7 | Predicted: Down | Actual: Up   | Prob: 0.4887
Day   8 | Predicted: Down | Actual: Down | Prob: 0.4887
Day   9 | Predicted: Down | Actual: Up   | Prob: 0.4887
Day  10 | Predicted: Down | Actual: Down | Prob: 0.4887
Day  11 | Predicted: Down | Actual: Down | Prob: 0.4887
Day  12 | Predicted: Down | Actual: Up   | Prob: 0.4887
Day  13 | Predicted: Down | Actual: Up   | Prob: 0.4887
Day  14 | Predicted: Down | Actual: Down | Prob: 0.4887
Day  15 | Predicted: Down | Actual: Up   | Prob: 0.4887
Day  16 | Predicted: Down | Actual: Down | Prob: 0.4887
Day  17 | Predicted: Do